In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
%cd "/content/drive/My Drive/data/audio"

/content/drive/My Drive/data/audio


In [0]:
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import numpy as np
import glob
import re
import math
import keras
from keras import regularizers
from keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from random import shuffle
from keras.datasets import mnist

Using TensorFlow backend.


In [0]:
songs = (re.search('.+?(?=\.wav)', x).group(0)
              for x in glob.iglob('*.wav')) 

#build spectrograms for conv net
spectrograms = []
for song in songs:
  sample_rate, samples = wavfile.read(song+'.wav')
  frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)
  spectrogram = spectrogram[:,10000:10500] #sample middle 2.5s of audio 
  #np.place(spectrogram, spectrogram == 0, 10**-10) #avoid div by 0 for log
  with open('../annotations/meter/' + song + '.meter', 'r') as file:
      meter = file.readline()

  spectrograms.append([song, spectrogram, meter])


In [0]:
import random
def load_data(spec_data, frac):
    #split = math.ceil(len(spec_data) * frac)
    split = 80
    
    random.shuffle(spec_data)
    
    x_train = np.array([x[1] for x in spec_data[:split]])
    y_train = np.array([x[2] for x in spec_data[:split]])
    
    x_test = np.array([x[1] for x in spec_data[split:]])
    y_test = np.array([x[2] for x in spec_data[split:]])
    
    return (x_train, y_train), (x_test, y_test)
 





In [0]:
from keras.applications.inception_v3 import InceptionV3, preprocess_input

(x_train, y_train), (x_test, y_test) = load_data(spectrograms, 0.7) #70% train-test split

#height and width of img
img_x, img_y = x_train.shape[1:]

#reshape x_train, x_test to 4D tensor - (samples, size1,size2,channels)
x_train = x_train.reshape((x_train.shape[0], img_x, img_y, 1))
x_test = x_test.reshape((x_test.shape[0], img_x, img_y, 1))

import cv2
x_train = cv2.merge((x_train,x_train,x_train)).reshape((x_train.shape[0], img_x, img_y, 3))
x_test = cv2.merge((x_test, x_test, x_test)).reshape((x_test.shape[0], img_x, img_y, 3))


#x_train = x_train.reshape((80, 129, 500, 1))
#x_train = cv2.merge((x_train, x_train, x_train))


#Does not work, why?
#stack greyscale img depth-wise to create img in rgb
#x_train = np.dstack((x_train, x_train, x_train))
#x_test = np.dstack((x_test, x_test, x_test))



#process input for compatibility with InceptionV3 model
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)
y_test = le.transform(y_test)


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 4)
y_test = keras.utils.to_categorical(y_test, 4)


#reshape y_train, y_test to (N,1) tensor for generator
#y_train = y_train.reshape(y_train.shape[0], 1)
#y_test = y_test.reshape(y_test.shape[0], 1)


print(np.sum(y_train, axis = 0)/len(y_train))
print(np.sum(y_test, axis = 0)/len(y_test))




[0.3    0.2875 0.2375 0.175 ]
[0.15789473 0.13157895 0.28947368 0.42105263]


In [0]:
#generator code for model.fit_generator()
def generator(features, labels, batch_size):
  # Create empty arrays to contain batch of features and labels

  batch_features = np.zeros((batch_size, img_x, img_y, 3))
  batch_labels = np.zeros((batch_size,1))

  while True:
   for i in range(batch_size):
     # choose random index in features
     index= np.random.choice(len(x_train))
     batch_features[i] = features[index]
     batch_labels[i] = labels[index]
   yield batch_features, keras.utils.to_categorical(batch_labels, num_classes=4)

In [0]:
from keras.layers import Input
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K


# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 4 classes
predictions = Dense(4, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)


for layer in model.layers:
   layer.trainable = True

# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), 
              loss='categorical_crossentropy',  metrics=['accuracy'])
       
# train the model on the new data for a few epochs
#model.fit_generator(generator(x_train, y_train, 5), steps_per_epoch=16, epochs=20)

model.fit(x_train, y_train,
          batch_size=5,
          epochs=200,
          verbose=1,
          validation_data=(x_test, y_test))



Train on 80 samples, validate on 38 samples
Epoch 1/200
80/80 [==============================] - 34s 426ms/step - loss: 1.4603 - acc: 0.2625 - val_loss: 1.4682 - val_acc: 0.1842
Epoch 2/200
80/80 [==============================] - 4s 53ms/step - loss: 1.4050 - acc: 0.2625 - val_loss: 1.5100 - val_acc: 0.2368
Epoch 3/200
80/80 [==============================] - 4s 53ms/step - loss: 1.3517 - acc: 0.3500 - val_loss: 1.4930 - val_acc: 0.2368
Epoch 4/200
80/80 [==============================] - 4s 53ms/step - loss: 1.3640 - acc: 0.3125 - val_loss: 1.4737 - val_acc: 0.2368
Epoch 5/200
80/80 [==============================] - 4s 53ms/step - loss: 1.3251 - acc: 0.3750 - val_loss: 1.4954 - val_acc: 0.1316
Epoch 6/200
80/80 [==============================] - 4s 53ms/step - loss: 1.3484 - acc: 0.2625 - val_loss: 1.4711 - val_acc: 0.2105
Epoch 7/200
80/80 [==============================] - 4s 53ms/step - loss: 1.3009 - acc: 0.4000 - val_loss: 1.4967 - val_acc: 0.2368
Epoch 8/200
80/80 [===========